In [1]:
from langchain_core.messages import HumanMessage
from typing import TypedDict, List, Annotated

# 定义系统状态结构
class AgentState(TypedDict):
    messages: Annotated[List[HumanMessage], "List of HumanMessage objects"]
    current_task: str
    sub_tasks: dict
    error_log: list

# 定义各领域智能体
def task_decomposer(state: AgentState) -> AgentState:
# 任务分解逻辑（示例）
    user_query = state["messages"][-1].content
    sub_tasks = {
    "transport": f"规划{user_query}中的交通方案",
    "hotel": f"推荐{user_query}期间的住宿",
    "attraction": f"建议{user_query}的景点路线"
        }
    state["sub_tasks"] = sub_tasks
    return state

def transport_planner(state: AgentState) -> AgentState:
    # 交通规划智能体（可接入实时API）
    task = state["sub_tasks"]["transport"]
    new_message = HumanMessage(content=f"交通方案：北京-上海高铁G12次，08:00-12:00。预算：553元")
    state["messages"].append(new_message)
    return state

def hotel_recommender(state: AgentState) -> AgentState:
    # 住宿推荐智能体（含价格筛选逻辑）
    new_message = HumanMessage(content="推荐酒店：上海中心大厦君悦\n• 外滩景观房 ￥1200/晚\n• 含双早+行政酒廊")
    state["messages"].append(new_message)
    return state

def attraction_advisor(state: AgentState) -> AgentState:
    # 景点规划智能体（含路线优化）
    new_message = HumanMessage(content="推荐路线：\nDay1: 外滩-城隍庙\nDay2: 迪士尼乐园\nDay3: 上海博物馆")
    state["messages"].append(new_message)
    return state

def error_handler(state: AgentState) -> AgentState:
    # 错误处理智能体
    error = state["error_log"][-1]
    new_message = HumanMessage(content=f"系统遇到问题：{error}，已启动备用方案...")
    state["messages"].append(new_message)
    return state

# 构建LangGraph工作流
# 假设 StateGraph 是一个有效的类，这里需要确保它已经被正确导入
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("task_decomposer", task_decomposer)
workflow.add_node("transport", transport_planner) 
workflow.add_node("hotel", hotel_recommender)
workflow.add_node("attraction", attraction_advisor)
workflow.add_node("error_handler", error_handler)

# 设置边关系
workflow.add_edge("task_decomposer", "transport")
workflow.add_edge("transport", "hotel)")

In [ ]:
from langchain_core.messages import HumanMessage
from typing import TypedDict, List, Annotated

# 定义系统状态结构
class AgentState(TypedDict):
    messages: Annotated[List[HumanMessage], "List of HumanMessage objects"]
    current_task: str
    sub_tasks: dict
    error_log: list

# 定义任务分解智能体
class TaskDecomposer:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 任务分解逻辑（示例）
        user_query = self.state["messages"][-1].content
        sub_tasks = {
            "transport": f"规划{user_query}中的交通方案",
            "hotel": f"推荐{user_query}期间的住宿",
            "attraction": f"建议{user_query}的景点路线"
        }
        self.state["sub_tasks"] = sub_tasks
        return self.state

# 定义交通规划智能体
class TransportPlanner:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 交通规划逻辑
        # task = self.state["sub_tasks"]["transport"]
        new_message = HumanMessage(content=f"交通方案：北京-上海高铁G12次，08:00-12:00。预算：553元")
        self.state["messages"].append(new_message)
        return self.state

# 定义住宿推荐智能体
class HotelRecommender:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 住宿推荐逻辑
        new_message = HumanMessage(content="推荐酒店：上海中心大厦君悦\n• 外滩景观房 ￥1200/晚\n• 含双早+行政酒廊")
        self.state["messages"].append(new_message)
        return self.state

# 定义景点规划智能体
class AttractionAdvisor:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 景点规划逻辑
        new_message = HumanMessage(content="推荐路线：\nDay1: 外滩-城隍庙\nDay2: 迪士尼乐园\nDay3: 上海博物馆")
        self.state["messages"].append(new_message)
        return self.state

# 定义错误处理智能体
class ErrorHandler:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 错误处理逻辑
        error = self.state["error_log"][-1]
        new_message = HumanMessage(content=f"系统遇到问题：{error}，已启动备用方案...")
        self.state["messages"].append(new_message)
        return self.state



draft_state = {
    "messages": [HumanMessage(content="帮我规划清明节上海3日游")],
    "current_task": "",
    "sub_tasks": {},
    "error_log": []
    }

agents= {
    "task_decomposer": TaskDecomposer(),  
    "transport": TransportPlanner(),
    "hotel": HotelRecommender(),
    "attraction": AttractionAdvisor(),
    "error_handler": ErrorHandler()
    }

# 构建LangGraph工作流
# 假设 StateGraph 是一个有效的类，这里需要确保它已经被正确导入
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("task_decomposer", agents["task_decomposer"].task)
workflow.add_node("transport", agents["transport"].task) 
workflow.add_node("hotel", agents["hotel"].task)
workflow.add_node("attraction", agents["attraction"].task)
workflow.add_node("error_handler", agents["error_handler"].task)

# 设置边关系
workflow.add_edge("task_decomposer", "transport")
workflow.add_edge("transport", "hotel")
workflow.add_edge("hotel", "attraction")

# 配置错误处理路径
workflow.add_conditional_edges(
    "transport",
    lambda state: "error" if "error" in state["error_log"] else "continue",
    {"error": "error_handler", "continue": "hotel"}
)

workflow.set_entry_point("task_decomposer")
workflow.set_finish_point("attraction")

# 编译执行
app = workflow.compile()

# 运行示例
result = app.invoke(draft_state)

print(result)

In [10]:
import os
import time
from langchain_core.messages import HumanMessage
from typing import TypedDict, List, Annotated, Callable, Dict
from langgraph.graph import StateGraph, END
import asyncio

# 定义系统状态结构
class AgentState(TypedDict):
    messages: Annotated[List[HumanMessage], "List of HumanMessage objects"]
    current_task: str
    sub_tasks: dict
    error_log: list

# 定义任务分解智能体
class TaskDecomposer:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 任务分解逻辑（示例）
        user_query = self.state["messages"][-1].content
        sub_tasks = {
            "transport": f"规划{user_query}中的交通方案",
            "hotel": f"推荐{user_query}期间的住宿",
            "attraction": f"建议{user_query}的景点路线"
        }
        self.state["sub_tasks"] = sub_tasks
        return self.state

# 定义交通规划智能体
class TransportPlanner:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 交通规划逻辑
        new_message = HumanMessage(content=f"交通方案：北京-上海高铁G12次，08:00-12:00。预算：553元")
        self.state["messages"].append(new_message)
        return self.state

# 定义住宿推荐智能体
class HotelRecommender:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 住宿推荐逻辑
        new_message = HumanMessage(content="推荐酒店：上海中心大厦君悦\n• 外滩景观房 ￥1200/晚\n• 含双早+行政酒廊")
        self.state["messages"].append(new_message)
        return self.state

# 定义景点规划智能体
class AttractionAdvisor:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 景点规划逻辑
        new_message = HumanMessage(content="推荐路线：\nDay1: 外滩-城隍庙\nDay2: 迪士尼乐园\nDay3: 上海博物馆")
        self.state["messages"].append(new_message)
        return self.state

# 定义错误处理智能体
class ErrorHandler:
    def __init__(self, state: AgentState):
        self.state = state

    def task(self) -> AgentState:
        # 错误处理逻辑
        error = self.state["error_log"][-1]
        new_message = HumanMessage(content=f"系统遇到问题：{error}，已启动备用方案...")
        self.state["messages"].append(new_message)
        return self.state

class TravelPlannerAgent:
    def __init__(self, task: dict):
        self.task = task
        self.state = self._initialize_state()

    def _initialize_state(self) -> AgentState:
        return {
            "messages": [HumanMessage(content=self.task.get("query", "默认查询"))],
            "current_task": "",
            "sub_tasks": {},
            "error_log": []
        }

    def _initialize_agents(self) -> Dict[str, Callable[[AgentState], AgentState]]:
        return {
            "task_decomposer": lambda state: TaskDecomposer(state).task(),
            "transport": lambda state: TransportPlanner(state).task(),
            "hotel": lambda state: HotelRecommender(state).task(),
            "attraction": lambda state: AttractionAdvisor(state).task(),
            "error_handler": lambda state: ErrorHandler(state).task()
        }

    def _create_workflow(self, agents: Dict[str, Callable[[AgentState], AgentState]]):
        workflow = StateGraph(AgentState)
        workflow.add_node("task_decomposer", agents["task_decomposer"])
        workflow.add_node("transport", agents["transport"])
        workflow.add_node("hotel", agents["hotel"])
        workflow.add_node("attraction", agents["attraction"])
        workflow.add_node("error_handler", agents["error_handler"])

        workflow.add_edge("task_decomposer", "transport")
        workflow.add_edge("transport", "hotel")
        workflow.add_edge("hotel", "attraction")

        workflow.add_conditional_edges(
            "transport",
            lambda state: "error" if "error" in state["error_log"] else "continue",
            {"error": "error_handler", "continue": "hotel"}
        )

        workflow.set_entry_point("task_decomposer")
        workflow.set_finish_point("attraction")
        return workflow

    def run(self):
        agents = self._initialize_agents()
        workflow = self._create_workflow(agents)
        app = workflow.compile()
        result = app.invoke(self.state)
        print(result)

# 示例运行
task = {"query": "帮我规划清明节上海3日游"}
agent = TravelPlannerAgent(task)
agent.run()

{'messages': [HumanMessage(content='帮我规划清明节上海3日游', additional_kwargs={}, response_metadata={}), HumanMessage(content='交通方案：北京-上海高铁G12次，08:00-12:00。预算：553元', additional_kwargs={}, response_metadata={}), HumanMessage(content='推荐酒店：上海中心大厦君悦\n• 外滩景观房 ￥1200/晚\n• 含双早+行政酒廊', additional_kwargs={}, response_metadata={}), HumanMessage(content='推荐路线：\nDay1: 外滩-城隍庙\nDay2: 迪士尼乐园\nDay3: 上海博物馆', additional_kwargs={}, response_metadata={})], 'current_task': '', 'sub_tasks': {'transport': '规划帮我规划清明节上海3日游中的交通方案', 'hotel': '推荐帮我规划清明节上海3日游期间的住宿', 'attraction': '建议帮我规划清明节上海3日游的景点路线'}, 'error_log': []}
